# MLOps implementation using SageMaker



This notebook covers the implementation part of the MLOps process inside SageMaker. Please note that this notebook is just for illustration purpose and will help you get started with the Implementation part. 

For this notebook, we will work with the dataset [Video Game Sales with Ratings](https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings) from Kaggle. 

The Machine Learning Problem Statement is that of predicting whether a given Video Game will become a hit in the market or not, based on the avaiable features. We are going to use the XGBoost Algorithm of different hyper-parameters for Model Selection process.

Once the models are created, we will leverage the Search API of SageMaker to choose which models needs to be deployed. 


In [ ]:
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt   
from IPython.display import Image                 
from IPython.display import display               
from sklearn.datasets import dump_svmlight_file   
from time import gmtime, strftime                 
import sys                                        
import math                                       
import json
import boto3
import sagemaker

session = sagemaker.Session()
region = session.boto_region_name
bucket ='mlopsbucket'#Replace it with the name of your s3 bucket. 
prefix = 'sagemaker/mlops-sample'
role = sagemaker.get_execution_role()

print('Bucket:\n{}'.format(bucket))

---
## Data


Before we proceed with the next steps, kindly create an account in Kaggle and download the Dataset and place it the S3 Bucket. Do not change the name of the dataset and retain the same after the CSV file download. 

In [ ]:
raw_data_filename = 'Video_Games_Sales_as_at_22_Dec_2016.csv'

s3 = boto3.resource('s3', region_name=region)
s3.Bucket(bucket).download_file(raw_data_filename, 'raw_data.csv')

data = pd.read_csv('./raw_data.csv')
pd.set_option('display.max_rows', 20) 
data.head()

In [ ]:
#Print the Number of records and variables present in the Dataset
data.shape

In [ ]:
#This command will print the features available in the Dataset
data.columns

Before proceeding further, we need to decide upon a target to predict. Video game development budgets can run into the tens of millions of dollars, so it is critical for game publishers to publish "hit" games to recoup their costs and make a profit. As a proxy for what constitutes a "hit" game, we will set a target of greater than 2 million units in global sales.

In [ ]:
data['y'] = (data['Global_Sales'] > 2)

In [ ]:
data['y'].value_counts()

With our target now defined, let's take a look at the imbalance between the "hit" and "not a hit" classes:

In [ ]:
plt.bar(['not a hit', 'hit'], data['y'].value_counts())
plt.show()

Not surprisingly, only a small fraction of games can be considered "hits" under our metric. Next, we'll choose features that have predictive power for our target. We'll begin by plotting review scores versus global sales to check our hunch that such scores have an impact on sales. Logarithmic scale is used for clarity.

In [ ]:
viz = data.filter(['User_Score','Critic_Score', 'Global_Sales'], axis=1)
viz['User_Score'] = pd.Series(viz['User_Score'].apply(pd.to_numeric, errors='coerce'))
viz['User_Score'] = viz['User_Score'].mask(np.isnan(viz["User_Score"]), viz['Critic_Score'] / 10.0)
viz.plot(kind='scatter', logx=True, logy=True, x='Critic_Score', y='Global_Sales')
viz.plot(kind='scatter', logx=True, logy=True, x='User_Score', y='Global_Sales')
plt.show()

Our intuition about the relationship between review scores and sales seems justified. We also note in passing that other relevant features can be extracted from the data set. For example, the ESRB rating has an impact since games with an "E" for everyone rating typically reach a wider audience than games with an age-restricted "M" for mature rating, though depending on another feature, the genre (such as shooter or action), M-rated games also can be huge hits. Our model hopefully will learn these relationships and others.  

Next, looking at the columns of features of this data set, we can identify several that should be excluded. For example, there are five columns that specify sales numbers: these numbers are directly related to the target we're trying to predict, so these columns should be dropped.  Other features may be irrelevant, such as the name of the game.

In [ ]:
data = data.drop(['Name', 'Year_of_Release', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Count', 'User_Count', 'Developer'], axis=1)

With the number of columns reduced, now is a good time to check how many columns are missing data:

In [ ]:
data.isnull().sum()

As noted in Kaggle's overview of this dataset, many review ratings are missing.  Unfortunately, since those are crucial features that we are relying on for our predictions, and there is no reliable way of imputing so many of them, we'll need to drop rows missing those features.

In [ ]:
data = data.dropna()

Now we need to resolve a problem we see in the User_Score column: it contains some 'tbd' string values, so it obviously is not numeric. User_Score is more properly a numeric rather than categorical feature, so we'll need to convert it from string type to numeric, and temporarily fill in NaNs for the tbds. Next, we must decide what to do with these new NaNs in the User_Score column. We've already thrown out a large number of rows, so if we can salvage these rows, we should. As a first approximation, we'll take the value in the Critic_Score column and divide by 10 since the user scores tend to track the critic scores (though on a scale of 0 to 10 instead of 0 to 100).  

In [ ]:
data['User_Score'] = data['User_Score'].apply(pd.to_numeric, errors='coerce')
data['User_Score'] = data['User_Score'].mask(np.isnan(data["User_Score"]), data['Critic_Score'] / 10.0)

Let's do some final preprocessing of the data, including converting the categorical features into numeric using the one-hot encoding method.

In [ ]:
if data['y'].dtype == bool:
    data['y'] = data['y'].apply(lambda y: 'yes' if y == True else 'no')
model_data = pd.get_dummies(data)

To help prevent overfitting the model, we'll randomly split the data into three groups. Specifically, the model will be trained on 70% of the data. It will then be evaluated on 20% of the data to give us an estimate of the accuracy we hope to have on "new" data. As a final testing dataset, the remaining 10% will be held out until the end.

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), 
                                          [int(0.7 * len(model_data)), int(0.9 * len(model_data))])   

Amazon SageMaker's version of XGBoost supports input data in both CSV and libSVM data format.  We'll use libSVM here, with features and the target variable provided as separate arguments. To avoid any misalignment issues due to random reordering, this split is done after the previous split in the above cell. As a last step before training, we'll copy the resulting files to S3 as input for Amazon SageMaker's hosted training.

In [ ]:
dump_svmlight_file(X=train_data.drop(['y_no', 'y_yes'], axis=1), y=train_data['y_yes'], f='train.libsvm')
dump_svmlight_file(X=validation_data.drop(['y_no', 'y_yes'], axis=1), y=validation_data['y_yes'], f='validation.libsvm')
dump_svmlight_file(X=test_data.drop(['y_no', 'y_yes'], axis=1), y=test_data['y_yes'], f='test.libsvm')

s3.Bucket(bucket).Object(prefix + '/train/train.libsvm').upload_file('train.libsvm')
s3.Bucket(bucket).Object(prefix + '/validation/validation.libsvm').upload_file('validation.libsvm')

s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='libsvm')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='libsvm')

---
## Train

For Demonstration Purpose, we are going to build an XGBoost Model available as a part of SageMaker's ML Algorithms with diferent hyperparameters.


In [ ]:
#max depth = 3
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(region, 'xgboost')

xgb_3 = sagemaker.estimator.Estimator(container,
                                    role, 
                                    base_job_name='xgboost3',
                                    train_instance_count=1, 
                                    train_instance_type='ml.c5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=session)

xgb_3.set_hyperparameters(max_depth=3,
                        eta=0.1,
                        subsample=0.5,
                        eval_metric='auc',
                        objective='binary:logistic',
                        scale_pos_weight=2.0,
                        num_round=100)


In [ ]:
#Running the XGBoost Model with Maximum Depth = 3
xgb_3.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
#xgboost with depth = 4
xgb_4 = sagemaker.estimator.Estimator(container,
                                    role, 
                                    base_job_name='xgboost4',
                                    train_instance_count=1, 
                                    train_instance_type='ml.c5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=session)

xgb_4.set_hyperparameters(max_depth=4,
                        eta=0.1,
                        subsample=0.5,
                        eval_metric='auc',
                        objective='binary:logistic',
                        scale_pos_weight=2.0,
                        num_round=100)

In [ ]:
#Running the XGBoost Model with Maximum Depth = 4
xgb_4.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
#xgboost with max-depth = 5
xgb_5= sagemaker.estimator.Estimator(container,
                                    role, 
                                    base_job_name='xgboost5',
                                    train_instance_count=1, 
                                    train_instance_type='ml.c5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=session)

xgb_5.set_hyperparameters(max_depth=5,
                        eta=0.1,
                        subsample=0.5,
                        eval_metric='auc',
                        objective='binary:logistic',
                        scale_pos_weight=2.0,
                        num_round=100)

In [ ]:
#Running the XGBoost Model with Max-Depth = 5
xgb_5.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
#Leveraging the Search API to identify the best models
import boto3

search_params={
   "MaxResults": 3,
   "Resource": "TrainingJob",
   "SearchExpression": { 
      "Filters": [{ 
            "Name": "TrainingJobName",
            "Operator": "Contains",
            "Value": "xgboost"
         }]}
}

smclient = boto3.client(service_name='sagemaker')
results = smclient.search(**search_params)

In [ ]:
#Creating a Dataframe of the Models and their Metrics to choose the best Model
import pandas

headers=["Training Job Name", "Training Job Status", "MaxDepth","Validation : AUC", "Training : AUC"]
rows=[]
for result in results['Results']: 
    trainingJob = result['TrainingJob']
    metrics = trainingJob['FinalMetricDataList']
    rows.append([trainingJob['TrainingJobName'],
     trainingJob['TrainingJobStatus'],
     trainingJob['HyperParameters']['max_depth'],
     metrics[[x['MetricName'] for x in  
     metrics].index('validation:auc')]['Value'],
     metrics[[x['MetricName'] for x in  
     metrics].index('train:auc')]['Value']
    ])

df = pandas.DataFrame(data=rows,columns=headers)

In [ ]:
df

---
## Host

Now that we've trained the XGBoost algorithm on our data, we can deploy the trained model to an Amazon SageMaker hosted endpoint with one simple line of code. We have identified from the above Dataframe that XGBoost with Maximum Depth = 5 has the highest AUC value. Hence we are going to deploy that model. 

In [ ]:
xgb_predictor_5 = xgb_5.deploy(initial_instance_count=1,
                           instance_type='ml.m5.xlarge')